# Qoperation objects
This notebook describes the concept of Qoperation in Quara and the related classes.

Quara supports operations such as State, Gate, Povm, etc.
These operations is called objects in Quara.
Two methods to generate objects.
- The typical objects can be generated from a module called xxx_typical. ("xxx" is a object name.)
- Users can also generate objects directly.


## Qoperation
Qoperation is the base class of these operations and has common properties and functions.

### common properties
The common properties are as follows:

- `is_physicality_required` - If this is True, this object must satisfy the physicality.
- `is_estimation_object` - If this is True, this object is for estimation.
- `on_para_eq_constraint` - If this is True, functions and algorithms work on the assumption that the parameters of this object satisfy the equality constraint. 
- `on_algo_eq_constraint` - If this is True, estimation algorithms use equality constraint. 
- `on_algo_ineq_constraint` - If this is True, estimation algorithms use inequality constraint. 
- `mode_proj_order` - the order in which the projection are performed, by default "eq_ineq".
   - "eq_ineq" - The projection are performed in the order of equality constraints, followed by inequality constraints.
   - "ineq_eq" - The projection are performed in the order of inequality constraints, followed by equality constraints.
- `eps_proj_physical` - Epsiron that is projection algorithm error threshold for being physical.
- `eps_truncate_imaginary_part` - The threshold to truncate imaginary part.

In [1]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.qoperation import QOperation

c_sys = generate_composite_system("qubit", 1)
qoperation = QOperation(c_sys)

print(qoperation.is_physicality_required)
print(qoperation.is_estimation_object)
print(qoperation.on_para_eq_constraint)
print(qoperation.on_algo_eq_constraint)
print(qoperation.on_algo_ineq_constraint)
print(qoperation.mode_proj_order)
print(qoperation.eps_proj_physical)
print(qoperation.eps_truncate_imaginary_part)

True
True
True
True
True
eq_ineq
1e-14
None


### functions to check properties
QOperation has functions to check properties.

- is_eq_constraint_satisfied - Whether the qoperation is satisfied equality constraints.
- is_ineq_constraint_satisfied - Whether the qoperation is satisfied inequality constraints.
- is_physical - Whether the qoperation is satisfied equality and inequality constraints.

is_eq_constraint_satisfied and is_ineq_constraint_satisfied are abstract functions and are implemented in a subclasses.

### projection functions
QOperation has projection functions.

- calc_proj_eq_constraint - Calculates the projection of QOperation on equal constraint.
- calc_proj_ineq_constraint - Calculates the projection of QOperation on inequal constraint.
- calc_proj_physical - Calculates the projection of QOperation on equal and inequal constraint.
- calc_proj_physical_with_var - Executes the algorithm of calc_proj_physical using variables. This functions is faster than calc_proj_physical.

calc_proj_physical and calc_proj_physical_with_var calculate the projection in the order specified by `mode_proj_order`.

### functions to transform parameters
QOperation has functions to transform parameters.

- to_stacked_vector - Transforms all parameters to a one-dimensional vector (a numpy array).
- to_var - Transforms parameters except for the part of the equality constraint to a one-dimensional vector (a numpy array).

These functions are abstract functions and are implemented in a subclasses.

### functions to generate special objects
QOperation has functions to generate special objects.

- generate_zero_obj - Generates object with all parameters are zero.
- generate_origin_obj - Generates origin object.

These functions are abstract functions and are implemented in a subclasses.

### supports arithmetic operations
Quara supports arithmetic operations between same type of QOperations.

- `__add__` - Calculates `a + b` for Qoperation a, b.
- `__sub__` - Calculates `a - b` for Qoperation a, b.
- `__mul__` - Calculates `a * b` for Qoperation a and number b.
- `__rmul__` - Calculates `a * b` for number a and Qoperation b.
- `__truediv__` - Calculates `a / b` for Qoperation a and number b.


## Notation
$I$ is the identity matrix.  
$X := \begin{pmatrix} 0 & 1 \\ 1 & 0 \\ \end{pmatrix}$, $Y := \begin{pmatrix} 0 & -i \\ i & 0 \\ \end{pmatrix}$, and $Z := \begin{pmatrix} 1 & 0 \\ 0 & -1 \\ \end{pmatrix}$ are Pauli matrices.  
$B$ is a basis $\{ I/\sqrt{2}, X/\sqrt{2}, Y/\sqrt{2}, Z/\sqrt{2} \}$, which is normalized by Hilbert-Schmidt inner product $\langle S, T\rangle := \text{Tr}[ST]$.

## State
The methods for generating a State includes the following:

- Generate from state_typical module
- Generate State object directly

Generate from state_typical module by specifying CompositeSystem and state name (ex. "z0").

In [2]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.state_typical import generate_state_from_name

c_sys = generate_composite_system("qubit", 1)
state = generate_state_from_name(c_sys, "z0")

Generate State object directly using CompositeSystem and a numpy array.

In [3]:
import numpy as np

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import State

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
vec = np.array([1, 0, 0, 1]) / np.sqrt(2)
state = State(c_sys, vec)

Quantum state (density matrix) can be represented by a linear combination of basis.
`vec` represents the coefficients of this linear combination in the form of a numpy array.  

Example.  
$z_0 = \begin{pmatrix} 1 & 0 \\ 0 & 0 \\ \end{pmatrix}$ can be represented by a linear combination of basis $1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} + 1/\sqrt{2} \cdot  Z/\sqrt{2}$. In this case `vec` of $z_0$ is $[ 1/\sqrt{2}, 0, 0, 1/\sqrt{2} ]$.

### specific properties
The property `vec` of State is a numpy array specified by the constructor argument `vec`.

In [4]:
state = State(c_sys, vec)
print(state.vec)

[0.70710678 0.         0.         0.70710678]


In [5]:
print(state.dim)
print(int(np.sqrt(len(vec))))

2
2


### functions to check properties
The is_eq_constraint_satisfied function returns True, if and only if $\text{Tr}[\rho] = 1$, where $\rho$ is a density matrix of State.

In [6]:
print(state.is_eq_constraint_satisfied())
print(np.trace(state.to_density_matrix()))

True
(0.9999999999999998+0j)


The is_ineq_constraint_satisfied function returns True, if and only if $\rho$ is positive semidifinite matrix, where $\rho$ is a density matrix of State.

In [7]:
print(state.is_eq_constraint_satisfied())
print(state.is_positive_semidefinite())

True
True


### projection functions
calc_proj_eq_constraint function calculates the projection of State on equal constraint.  
This function replaces the first element of `vec` with $1/\sqrt{d}$, where $d$ is `dim`.

In [8]:
vec = np.array([1.0, 2.0, 3.0, 4.0])
state = State(c_sys, vec, is_physicality_required=False)
proj_state = state.calc_proj_eq_constraint()
print(proj_state.vec)

[0.70710678 2.         3.         4.        ]


calc_proj_ineq_constraint function calculates the projection of State on equal constraint as follows:  

- Executes singular value decomposition on the density matrix $\rho$ of state, $\rho = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
- $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
- $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
- $\rho^{\prime} = U \Lambda^{\prime} U^{\dagger}$
- The projection of State is $|\rho^{\prime} \rangle\rangle$.

In [9]:
vec = np.sqrt(2) * np.array([0, 0, 0, 1])
state = State(c_sys, vec, is_physicality_required=False)
print(state.to_density_matrix())
proj_state = state.calc_proj_ineq_constraint()
print(proj_state.to_density_matrix())
print(proj_state.vec)

[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[0.70710678 0.         0.         0.70710678]


### functions to transform parameters
to_stacked_vector function returns a one-dimensional numpy array of all variables. This is equal to `vec`.


In [10]:
print(state.to_stacked_vector())
print(state.vec)

[0.         0.         0.         1.41421356]
[0.         0.         0.         1.41421356]


If `on_para_eq_constraint` is True, then the first element of `vec` is equal to $1/\sqrt{d}$, where $d$ is `dim`. Thus, State is characterized by the second and subsequent elements of `vec`.  
Therefore, to_var function returns the second and subsequent elements of `vec`, where `on_para_eq_constraint` is True.

In [50]:
vec = np.array([1, 0, 0, 1]) / np.sqrt(2)

In [51]:
# on_para_eq_constraint=True
state = State(c_sys, vec, on_para_eq_constraint=True)
print(state.to_var())

[0.         0.         0.70710678]


In [52]:
# on_para_eq_constraint=False
state = State(c_sys, vec, on_para_eq_constraint=False)
print(state.to_var())

[0.70710678 0.         0.         0.70710678]


### functions to generate special objects

In [14]:
zero_state = state.generate_zero_obj()
print(zero_state.vec)
origin_state = state.generate_origin_obj()
print(origin_state.vec)

[0. 0. 0. 0.]
[0.70710678 0.         0.         0.        ]


### supports arithmetic operations

In [15]:
vec1 = np.array([1.0, 2.0, 3.0, 4.0])
state1 = State(c_sys, vec1, is_physicality_required=False)
vec2 = np.array([5.0, 6.0, 7.0, 8.0])
state2 = State(c_sys, vec2, is_physicality_required=False)

print((state1 + state2).vec)
print((state1 - state2).vec)
print((2 * state1).vec)
print((state1 * 2).vec)
print((state1 / 2).vec)

[ 6.  8. 10. 12.]
[-4. -4. -4. -4.]
[2. 4. 6. 8.]
[2. 4. 6. 8.]
[0.5 1.  1.5 2. ]


### calc_gradient functions
Calculates gradient of State with variable index.

In [1]:
grad_state = state.calc_gradient(0)
print(grad_state.vec)

NameError: name 'state' is not defined

### convert_basis function
Returns `vec` converted to the specified basis.

In [17]:
from quara.objects.matrix_basis import get_comp_basis

state = generate_state_from_name(c_sys, "z0")
converted_vec = state.convert_basis(get_comp_basis())
print(converted_vec)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


### to_density_matrix function
Returns density matrix of State. 

In [18]:
state = generate_state_from_name(c_sys, "z0")
print(state.to_density_matrix())
print(state.to_density_matrix_with_sparsity())

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### some utility functions

In [19]:
print(state.is_trace_one())
print(state.is_hermitian())
print(state.is_positive_semidefinite())
print(state.calc_eigenvalues())

True
True
True
[0.9999999999999998, 0.0]


## state_typical
generate_state_object_from_state_name_object_name function in state_typical module can easily generate objects related to State.  
The generate_state_object_from_state_name_object_name function has the following arguments:

- The string that can be specified for `state_name` can be checked by executing the get_state_names function. The tensor product of state_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "pure_state_vector" - vector of pure state.
  - "density_mat" - density matrix.
  - "density_matrix_vector" - vectorized density matrix.
  - "state" - State object.
- `c_sys` - CompositeSystem of objects related to State. Specify when `object_name` is "density_matrix_vector" or "state".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [20]:
from quara.objects.state_typical import (
    get_state_names,
    generate_state_object_from_state_name_object_name,
)

print(get_state_names())

['x0', 'x1', 'y0', 'y1', 'z0', 'z1', 'a', 'bell_phi_plus', 'bell_phi_minus', 'bell_psi_plus', 'bell_psi_minus', 'x0_x0', 'x0_x1', 'x0_y0', 'x0_y1', 'x0_z0', 'x0_z1', 'x0_a', 'x1_x0', 'x1_x1', 'x1_y0', 'x1_y1', 'x1_z0', 'x1_z1', 'x1_a', 'y0_x0', 'y0_x1', 'y0_y0', 'y0_y1', 'y0_z0', 'y0_z1', 'y0_a', 'y1_x0', 'y1_x1', 'y1_y0', 'y1_y1', 'y1_z0', 'y1_z1', 'y1_a', 'z0_x0', 'z0_x1', 'z0_y0', 'z0_y1', 'z0_z0', 'z0_z1', 'z0_a', 'z1_x0', 'z1_x1', 'z1_y0', 'z1_y1', 'z1_z0', 'z1_z1', 'z1_a', 'a_x0', 'a_x1', 'a_y0', 'a_y1', 'a_z0', 'a_z1', 'a_a', 'ghz', 'werner', 'x0_x0_x0', 'x0_x0_x1', 'x0_x0_y0', 'x0_x0_y1', 'x0_x0_z0', 'x0_x0_z1', 'x0_x0_a', 'x0_x1_x0', 'x0_x1_x1', 'x0_x1_y0', 'x0_x1_y1', 'x0_x1_z0', 'x0_x1_z1', 'x0_x1_a', 'x0_y0_x0', 'x0_y0_x1', 'x0_y0_y0', 'x0_y0_y1', 'x0_y0_z0', 'x0_y0_z1', 'x0_y0_a', 'x0_y1_x0', 'x0_y1_x1', 'x0_y1_y0', 'x0_y1_y1', 'x0_y1_z0', 'x0_y1_z1', 'x0_y1_a', 'x0_z0_x0', 'x0_z0_x1', 'x0_z0_y0', 'x0_z0_y1', 'x0_z0_z0', 'x0_z0_z1', 'x0_z0_a', 'x0_z1_x0', 'x0_z1_x1', 'x0_z

### pure_state_vector

In [21]:
vec = generate_state_object_from_state_name_object_name("z0", "pure_state_vector")
print(vec)

[1.+0.j 0.+0.j]


### density_mat

In [22]:
mat = generate_state_object_from_state_name_object_name("z0", "density_mat")
print(mat)

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### density_matrix_vector

In [23]:
c_sys = generate_composite_system("qubit", 1)
vec = generate_state_object_from_state_name_object_name("z0", "density_matrix_vector", c_sys=c_sys)
print(vec)

[0.70710678 0.         0.         0.70710678]


### state

In [24]:
c_sys = generate_composite_system("qubit", 1)
state = generate_state_object_from_state_name_object_name("z0", "state", c_sys=c_sys)
print(state)

Type:
State

Dim:
2

Vec:
[0.70710678 0.         0.         0.70710678]


## Povm
The methods for generating a State includes the following:

- Generate from povm_typical module
- Generate Povm object directly

Generate from povm_typical module by specifying CompositeSystem and povm name (ex. "z").

In [25]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.povm_typical import generate_povm_from_name

c_sys = generate_composite_system("qubit", 1)
povm = generate_povm_from_name("z", c_sys)

Generate Povm object directly using CompositeSystem and a list of numpy array.

In [26]:
import numpy as np

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import Povm

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
vec1 = np.array([1, 0, 0, 1]) / np.sqrt(2)
vec2 = np.array([1, 0, 0, -1]) / np.sqrt(2)
vecs = [vec1, vec2]
povm = Povm(c_sys, vecs)

Povm can be represented by a list of POVM element as `vecs`.  
Each emelemts is a linear combination of basis and represents the coefficients of this linear combination in the form of a numpy array.

Example:  
Povm elements $\Pi_0 = \begin{pmatrix} 1 & 0 \\ 0 & 0 \\ \end{pmatrix}$, $\Pi_1 = \begin{pmatrix} 0 & 0 \\ 0 & 1 \\ \end{pmatrix}$.  
Povm $\Pi = \{ \Pi_x \}_{x=0,1}$. index $x$ is called outcomes.  
We can write $\Pi_0= 1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} + 1/\sqrt{2} \cdot  Z/\sqrt{2}$ and  $\Pi_1= 1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} - 1/\sqrt{2} \cdot  Z/\sqrt{2}$.  
In this case `vecs` of $\Pi$ is a list of $[ 1/\sqrt{2}, 0, 0, 1/\sqrt{2} ]$ and $[ 1/\sqrt{2}, 0, 0, -1/\sqrt{2} ]$.

### specific properties
The property `vecs` of Povm is a list of a numpy array specified by the constructor argument `vecs`.

In [27]:
povm = Povm(c_sys, vecs)
print(povm.vecs)

(array([0.70710678, 0.        , 0.        , 0.70710678]), array([ 0.70710678,  0.        ,  0.        , -0.70710678]))


The property `dim` of Povm is a square root of the size of element of `vecs`.

In [28]:
print(povm.dim)
print(int(np.sqrt(len(vecs[0]))))

2
2


### functions to check properties
The is_eq_constraint_satisfied function returns True, if and only if $\sum_x \Pi_x = I$, where $\Pi_x$ are Povm elements.

In [29]:
from operator import add
from functools import reduce

print(povm.is_eq_constraint_satisfied())
print(reduce(add, povm.matrices()))

True
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


The is_ineq_constraint_satisfied function returns True, if and only if all $\Pi_x$ are positive semidifinite matrices.

In [30]:
print(povm.is_eq_constraint_satisfied())
print(povm.matrix(0))
print(povm.matrix(1))

True
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


### projection functions

### functions to transform parameters
to_stacked_vector function returns a one-dimensional numpy array of all variables. This is equal to a vector combining elements of `vecs`.

In [31]:
print(povm.to_stacked_vector())
print(povm.vecs)

[ 0.70710678  0.          0.          0.70710678  0.70710678  0.
  0.         -0.70710678]
(array([0.70710678, 0.        , 0.        , 0.70710678]), array([ 0.70710678,  0.        ,  0.        , -0.70710678]))


If Povm $\Pi = \{ \Pi_0, \dots , \Pi_{m-1} \}$ and `on_para_eq_constraint` is True, then the last element of Povm $\Pi_{m-1}$ is equal to $I - \sum_{x=0}^{m-2} \Pi_x$. Thus, Povm is characterized by `vecs[0]`,..., `vecs[m-2]`.  
Therefore, to_var function returns a vector combining elements of of `vecs[0]`,..., `vecs[m-2]`, where `on_para_eq_constraint` is True.

In [47]:
vec1 = np.array([1, 0, 0, 1]) / np.sqrt(2)
vec2 = np.array([1, 0, 0, -1]) / np.sqrt(2)
vecs = [vec1, vec2]

In [48]:
# on_para_eq_constraint=True
povm = Povm(c_sys, vecs, on_para_eq_constraint=True)
print(povm.to_var())

[0.70710678 0.         0.         0.70710678]


In [49]:
# on_para_eq_constraint=False
povm = Povm(c_sys, vecs, on_para_eq_constraint=False)
print(povm.to_var())

[ 0.70710678  0.          0.          0.70710678  0.70710678  0.
  0.         -0.70710678]


### functions to generate special objects

In [34]:
zero_povm = povm.generate_zero_obj()
print(zero_povm.vecs)
origin_povm = povm.generate_origin_obj()
print(origin_povm.vecs)

(array([0., 0., 0., 0.]), array([0., 0., 0., 0.]))
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))


### supports arithmetic operations

In [35]:
vec11 = np.array([1.0, 2.0, 3.0, 4.0])
vec12 = np.array([5.0, 6.0, 7.0, 8.0])
povm1 = Povm(c_sys, [vec11, vec12], is_physicality_required=False)
vec21 = np.array([9.0, 10.0, 11.0, 12.0])
vec22 = np.array([13.0, 14.0, 15.0, 16.0])
povm2 = Povm(c_sys, [vec21, vec22], is_physicality_required=False)

print((povm1 + povm2).vecs)
print((povm1 - povm2).vecs)
print((2 * povm1).vecs)
print((povm1 * 2).vecs)
print((povm1 / 2).vecs)

(array([10., 12., 14., 16.]), array([18., 20., 22., 24.]))
(array([-8., -8., -8., -8.]), array([-8., -8., -8., -8.]))
(array([2., 4., 6., 8.]), array([10., 12., 14., 16.]))
(array([2., 4., 6., 8.]), array([10., 12., 14., 16.]))
(array([0.5, 1. , 1.5, 2. ]), array([2.5, 3. , 3.5, 4. ]))


### calc_gradient functions
Calculates gradient of Povm with variable index.

In [36]:
grad_povm = povm.calc_gradient(0)
print(grad_povm.vecs)

(array([1., 0., 0., 0.]), array([0., 0., 0., 0.]))


### convert_basis function
Returns `vecs` converted to the specified basis.

In [37]:
from quara.objects.matrix_basis import get_comp_basis

povm = generate_povm_from_name("z", c_sys)
converted_vecs = povm.convert_basis(get_comp_basis())
print(converted_vecs)

[array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]), array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]


### generate_mprocess function
Generates MProcess from this Povm.

In [38]:
print(povm.generate_mprocess())

16it [00:00, 296.31it/s]Type:
MProcess

Dim:
2

HSs:
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]

ModeSampling:
False



### some utility functions

In [39]:
print(povm.vec(0))
print(povm.matrices())
print(povm.matrix(0))
print(povm.is_hermitian())
print(povm.is_positive_semidefinite())
print(povm.is_identity_sum())
print(povm.calc_eigenvalues(0))

[0.70710678 0.         0.         0.70710678]
[matrix([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]), matrix([[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]])]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
True
True
True
[0.9999999999999998, 0.0]


## povm_typical
generate_povm_object_from_povm_name_object_name function in povm_typical module can easily generate objects related to Povm.  
The generate_povm_object_from_povm_name_object_name function has the following arguments:

- The string that can be specified for `povm_name` can be checked by executing the get_povm_names function. The tensor product of povm_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "pure_state_vectors" - list of vector of pure states.
  - "matrices" - matrices of vector of pure states.
  - "vectors" - list of vectorized matrices.
  - "povm" - Povm object.
- `c_sys` - CompositeSystem of objects related to Povm. Specify when `object_name` is "povm".
- `basis` - MatrixBasis of objects related to Povm. Specify when `object_name` is "vectors".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [40]:
from quara.objects.povm_typical import (
    get_povm_names,
    generate_povm_object_from_povm_name_object_name,
)

print(get_povm_names())

['x', 'y', 'z', 'bell', 'x_x', 'x_y', 'x_z', 'y_x', 'y_y', 'y_z', 'z_x', 'z_y', 'z_z', 'x_x_x', 'x_x_y', 'x_x_z', 'x_y_x', 'x_y_y', 'x_y_z', 'x_z_x', 'x_z_y', 'x_z_z', 'y_x_x', 'y_x_y', 'y_x_z', 'y_y_x', 'y_y_y', 'y_y_z', 'y_z_x', 'y_z_y', 'y_z_z', 'z_x_x', 'z_x_y', 'z_x_z', 'z_y_x', 'z_y_y', 'z_y_z', 'z_z_x', 'z_z_y', 'z_z_z', '01x3', '01y3', 'z3', 'z2', '02x3', '02y3', '12x3', '12y3', '01x3_01x3', '01x3_01y3', '01x3_z3', '01x3_z2', '01x3_02x3', '01x3_02y3', '01x3_12x3', '01x3_12y3', '01y3_01x3', '01y3_01y3', '01y3_z3', '01y3_z2', '01y3_02x3', '01y3_02y3', '01y3_12x3', '01y3_12y3', 'z3_01x3', 'z3_01y3', 'z3_z3', 'z3_z2', 'z3_02x3', 'z3_02y3', 'z3_12x3', 'z3_12y3', 'z2_01x3', 'z2_01y3', 'z2_z3', 'z2_z2', 'z2_02x3', 'z2_02y3', 'z2_12x3', 'z2_12y3', '02x3_01x3', '02x3_01y3', '02x3_z3', '02x3_z2', '02x3_02x3', '02x3_02y3', '02x3_12x3', '02x3_12y3', '02y3_01x3', '02y3_01y3', '02y3_z3', '02y3_z2', '02y3_02x3', '02y3_02y3', '02y3_12x3', '02y3_12y3', '12x3_01x3', '12x3_01y3', '12x3_z3', '12x3

### pure_state_vectors

In [41]:
vecs = generate_povm_object_from_povm_name_object_name("z", "pure_state_vectors")
print(vecs)

[array([1.+0.j, 0.+0.j]), array([0.+0.j, 1.+0.j])]


### matrices

In [43]:
matrices = generate_povm_object_from_povm_name_object_name("z", "matrices")
print(matrices)

[array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]]), array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])]


### vectors

In [44]:
basis = get_normalized_pauli_basis(1)
vectors = generate_povm_object_from_povm_name_object_name("z", "vectors", basis=basis)
print(vectors)

[array([0.70710678, 0.        , 0.        , 0.70710678]), array([ 0.70710678,  0.        ,  0.        , -0.70710678])]


### povm

In [46]:
c_sys = generate_composite_system("qubit", 1)
povm = generate_povm_object_from_povm_name_object_name("z", "povm", c_sys=c_sys)
print(povm)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
